![image.png](https://raw.githubusercontent.com/fjvarasc/DSPXI/master/figures/python_logo.png)

# Excel Lab

In this example, a fictional company called ABCD Catering has recorded sales and order history for 2009 in their corporate ERP system. ABCD Catering provides catering services to leading Silicon Valley companies, providing the best in hamburgers, hot dogs, churros, sodas and other comfort. Your boss has asked you to examine this data and answer some questions and produce charts representing some of the data:

* What were the total sales in each of the last four quarters?
* What are the sales for each food item in each quarter?
* Who were the top 10 customers for ABCD catering in Q1?
* Who was the highest producing sales rep for the year?
* What food item had the highest unit sales in Q4?

Generating this information typically involves running five separate reports in the system. Since your boss is looking for this same information at the end of each quarter, you want to simplify your life and your bosses by automating the report. Using Python and Excel, you can download a spreadsheet copy of the raw data, process it, generate the key figures and charts and save them to a spreadsheet.

Take a look at the data in ABCDCatering.xls:

![image.png](https://raw.githubusercontent.com/fjvarasc/DSPXI/master/figures/20091102_original.png)

The spreadsheet contains some header information, then a large table of records for each order. Each record contains the fiscal year and quarter, food item, company name, order data, sales representative, booking and order quantity for each order. The data needs some work before you can use it in a pivot table. First, the data in rows 1 through 11 must be ignored, it’s meaningless for the pivot table. Also, some columns do not have a proper header and must be corrected before the data can be used. The good news is that after some minor massaging, this data will be ideally suited for processing with a pivot table in Excel. Close the spreadsheet and get ready to build the reports.

The program begins with the standard boilerplate: import the win32 module and start Excel.

## Start Excel

In [ ]:
import win32com.client as win32
import sys
import os

excel = win32.gencache.EnsureDispatch('Excel.Application')
excel.Visible = True

Next, open the spreadsheet ABCDCatering.xls with some exception handling. The try/except clause attempts to open the file with the Workbooks.Open() method, and exits gracefully if the file is missing or some other problem occurred. Lastly, the variable ws is set to the spreadsheet containing the data.

## Open Workbook

In [ ]:
default_path =  os.getcwd() + '\\' 

try:
    wb = excel.Workbooks.Open(default_path + 'ABCDCatering.xls')
except:
    print "Failed to open spreadsheet ABCDCatering.xls"
    sys.exit(1)
ws = wb.Sheets('Sheet1')

An easy way to load the entire spreadsheet into Python is the UsedRange method. The following command:

## Load Sheet Data in python

In [ ]:
xldata = ws.UsedRange.Value

The above code grabs all the data in the Sheet1 worksheet and copies it into a tuple named xldata. Once inside Python, the data can be manipulated and placed back into the spreadsheet with minimal calls to the COM interface, resulting in faster, more efficient processing.

## Check Data

To delete rows, add columns and do other operations on the data, it must be converted to or copied to a list. The approach used here is to examine the data row by row, discarding the non essential header rows and copying everything else to a new list. The first step is to remove the rows that are not part of the column header row or record data. If you are using Python to generate the program interactively, Let's can investigate the data in the xldata tuple and display the data for the first record (xldata[0]) and header record (xldata[11]):

In [ ]:
xldata[0]

In [ ]:
xldata[11]

The length of both rows is 13, though xldata[0] contains many elements with a value of None. The following code checks the length of the data and skips any rows shorter then 13 fields or rows that contain None in the last field. Note that this code assumes that the actual data in the table always contains complete records, true in this dataset but you should always understand the characteristics of the data you’re working on.

## Fix Data

In [ ]:
newdata = []
for row in xldata:
    if row[-1] is not None and len(row) == 13:
        newdata.append(row)

The `newdata` list now contains the header and data rows from the spreadsheet, but the header row is still not complete. All column headers must contain text in order to use this data in a pivot table. Unfortunately, the spreadsheet downloads produced by the ERP system have the column label over the numberical identifier for the item, while the text column header is blank. You can see that for the “Food” and “Company” data below.

![image.png](https://raw.githubusercontent.com/fjvarasc/DSPXI/master/figures/20091102_foodcompany.png)

One approach that works for this data is to scan the header and insert a column header based on the contents of the previous column. For example, the label for column F could be “Company Name”, created by simply appending the text ” Name” to the column header “Company” from the prior column. Using this simple algorithm, the column header row can be filled out and the spreadsheet made ready for pivot table conversion. A more complex lookup could be used as well, but the simple algorithm described here will scale if new fields are added to the report.

In [ ]:
for i,field in enumerate(newdata[0]):
    if field is None:
        newdata[0][i] = lasthdr + " Name"
    else:
        lasthdr = newdata[0][i]

Now the data is ready for insertion back into the spreadsheet. To enable comparison between the new data set and the original, create a new sheet in the workbook, write the data to the new sheet and autofit the columns.

Now the data is ready for insertion back into the spreadsheet. To enable comparison between the new data set and the original, create a new sheet in the workbook, write the data to the new sheet and autofit the columns.

In [ ]:
wsnew = wb.Sheets.Add()
wsnew.Range(wsnew.Cells(1,1),wsnew.Cells(len(newdata),len(newdata[0]))).Value = newdata
wsnew.Columns.AutoFit()

The last step is to save the worksheet to a new file and quit Excel.

In [ ]:
wb.SaveAs('newABCDCatering.xlsx')
excel.Application.Quit()
#wb.SaveAs('newABCDCatering.xlsx',win32.constants.xlOpenXMLWorkbook)

## Check the output

After running the script, open the file newABCDCatering.xlsx or newABCDCatering.xls and view the contents. Note that the extraneous header information has been removed and blank column header information has been inserted programmatically as described earlier.

![image.png](https://raw.githubusercontent.com/fjvarasc/DSPXI/master/figures/20091102_exceloutput.png)

The new spreadsheet is ready for use in a pivot table :)

## Build the APP & Generate Reports

Pivot tables are an easy-to-use tool to derive some basic business intelligence from your data. As discussed last time, there are occasions when you’ll need to do interactive data mining by changing column and row fields. But in my experience, it’s handy to have my favorite reports built automatically, with the reports ready to go as soon as I open the spreadsheet. In this post I’ll develop and explain the code to create a set of pivot tables automatically in worksheet.

The goal of this exercise is to automate the generation of pivot tables and save them to a new Excel file.
![image.png](https://raw.githubusercontent.com/fjvarasc/DSPXI/master/figures/20091123_reports.png)

load newABCDCatering.xls from the previous step and record the macro to create this simple pivot table showing Net Bookings by Sales Rep and Food Name for the last four quarters as below.
![image.png](https://raw.githubusercontent.com/fjvarasc/DSPXI/master/figures/20091123_setup.png)

The captured macro should be similar to this.

```vbscript
'
' Macro2 Macro
'

'
    Selection.CurrentRegion.Select
    Sheets.Add
    ActiveWorkbook.PivotCaches.Create(SourceType:=xlDatabase, SourceData:= _
        "Sheet2!R1C1:R791C13", Version:=xlPivotTableVersion10).CreatePivotTable _
        TableDestination:="Sheet3!R3C1", TableName:="PivotTable1", DefaultVersion _
        :=xlPivotTableVersion10
    Sheets("Sheet3").Select
    Cells(3, 1).Select
    With ActiveSheet.PivotTables("PivotTable1").PivotFields("Fiscal Year")
        .Orientation = xlPageField
        .Position = 1
    End With
    With ActiveSheet.PivotTables("PivotTable1").PivotFields("Fiscal Quarter")
        .Orientation = xlColumnField
        .Position = 1
    End With
    With ActiveSheet.PivotTables("PivotTable1").PivotFields("Sales Rep Name")
        .Orientation = xlRowField
        .Position = 1
    End With
    With ActiveSheet.PivotTables("PivotTable1").PivotFields("Food Name")
        .Orientation = xlRowField
        .Position = 2
    End With
    ActiveSheet.PivotTables("PivotTable1").AddDataField ActiveSheet.PivotTables( _
        "PivotTable1").PivotFields("Net Booking"), "Sum of Net Booking", xlSum
End Sub
```

Looking at the macro, you see lines specifying the Orientation of the field name, such as `.Orientation = xlRowField` and `.Orientation = xlColumnField`. A pivot table has four basic areas for fields:

* Report Filter (.Orientation = xlPageField)
* Column area (.Orientation = xlColumnField)
* Row area (.Orientation = xlRowField)
* Values area (PivotTables().AddDataField())

Each of these supports multiple fields (column fields for `Sales Rep Name` and `Food Name` were added in the example). The ordering of the fields changes the appearance of the table.

A general pattern should be apparent in this macro. First, the pivot table is created with the `ActiveWorkbook.PivotCaches.Create()` statement. Next, the columns and rows are configured with a series of `ActiveSheet.PivotTables("PivotTable1").PivotFields()` statements. Finally, the field used in the Values section of the table is configured using the `ActiveSheet.PivotTables("PivotTable1").AddDataField` statement. The general purpose function will need to contain all of these constructs. Note the parts that can’t be hard-coded: the source of the data, `"Sheet2!R1C1:R791C13"`, and destination for the table, `"Sheet3!R3C1"` need to be determined based on the characteristics of the source data and can’t be hard coded in the general solution.

In Python, this pattern can be reduced to the following loop that covers fields for the Report Filter, Columns and Rows:

In [ ]:
def addpivot(wb,sourcedata,title,filters=(),columns=(),
             rows=(),sumvalue=(),sortfield=""):
    """Build a pivot table using the provided source location data
    and specified fields
    """
    ...
    for fieldlist,fieldc in ((filters,win32c.xlPageField),
                            (columns,win32c.xlColumnField),
                            (rows,win32c.xlRowField)):
        for i,val in enumerate(fieldlist):
            wb.ActiveSheet.PivotTables(tname).PivotFields(val).Orientation = fieldc
        wb.ActiveSheet.PivotTables(tname).PivotFields(val).Position = i+1
    ...

Processing the Values field is more or less copied from the Visual Basic. To keep things simple in this example, this code is limited to adding “Sum of” values only, and doesn’t handle other Summarize Value functions such as Count, Min, Max, etc.

In [ ]:
wb.ActiveSheet.PivotTables(tname).AddDataField(
    wb.ActiveSheet.PivotTables(tname).PivotFields(sumvalue[7:]),
    sumvalue,
    win32c.xlSum)

The actual values for filters, columns and rows in the function are defined in the call to the function. The complete function creates a new sheet within the workbook, then adds an empty pivot table to the sheet and builds the table using the field information provided. For example, to answer the question: What were the total sales in each of the last four quarters?, the pivot table is built with the following call to the addpivot function:

In [ ]:
# What were the total sales in each of the last four quarters?
addpivot(wb,src,title="Sales by Quarter",filters=(),columns=(),rows=("Fiscal Quarter",),sumvalue="Sum of Net Booking",
         sortfield=())

the above code defines a pivot table using the row header “Fiscal Quarter” and data value “Sum of Net Booking”. The title “Sales by Quarter” is used to name the sheet itself.

To make the output spreadsheet more understandable, the title parameter passed into the function and used as a title in each worksheet and as the tab name.

![image.png](https://raw.githubusercontent.com/fjvarasc/DSPXI/master/figures/20091123_titletabsbq.png)

The complete script is shown below:

* This script has been modified to run on both Excel 2007 or later and Excel 2003 and has been tested on those versions.
* Adding pivot tables increases the size of the output Excel file, which can be mitigated by disabling caching of pivot table data. `PivotTables(tname).SaveData = False`, this will reduce the size of the output Excel file, but will require that the pivot table be refreshed before use by clicking on Refresh Data on the PivotTable toolbar.



In [ ]:
#
# erpdatapivot.py:
# Load raw EPR data, clean up header info and
# build 5 pivot tables
#
import win32com.client as win32
win32c = win32.constants
import sys
import itertools
tablecount = itertools.count(1)

def addpivot(wb,sourcedata,title,filters=(),columns=(),
             rows=(),sumvalue=(),sortfield=""):
    """Build a pivot table using the provided source location data
    and specified fields
    """
    newsheet = wb.Sheets.Add()
    newsheet.Cells(1,1).Value = title
    newsheet.Cells(1,1).Font.Size = 16

    # Build the Pivot Table
    tname = "PivotTable%d"% next(tablecount)#tablecount.next()

    pc = wb.PivotCaches().Add(SourceType=win32c.xlDatabase,
                                 SourceData=sourcedata)
    pt = pc.CreatePivotTable(TableDestination="%s!R4C1"%newsheet.Name,
                             TableName=tname,
                             DefaultVersion=win32c.xlPivotTableVersion10)
    wb.Sheets(newsheet.Name).Select()
    wb.Sheets(newsheet.Name).Cells(3,1).Select()
    for fieldlist,fieldc in ((filters,win32c.xlPageField),
                            (columns,win32c.xlColumnField),
                            (rows,win32c.xlRowField)):
        for i,val in enumerate(fieldlist):
            wb.ActiveSheet.PivotTables(tname).PivotFields(val).Orientation = fieldc
            wb.ActiveSheet.PivotTables(tname).PivotFields(val).Position = i+1

    wb.ActiveSheet.PivotTables(tname).AddDataField(
        wb.ActiveSheet.PivotTables(tname).PivotFields(sumvalue[7:]),
        sumvalue,
        win32c.xlSum)
    if len(sortfield) != 0:
        wb.ActiveSheet.PivotTables(tname).PivotFields(sortfield[0]).AutoSort(sortfield[1], sumvalue)
    newsheet.Name = title

    # Uncomment the next command to limit output file size, but make sure
    # to click Refresh Data on the PivotTable toolbar to update the table
    # newsheet.PivotTables(tname).SaveData = False

    return tname

def runexcel():
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    excel.Visible = True
    try:
        wb = excel.Workbooks.Open('ABCDCatering.xls')
    except:
        print ("Failed to open spreadsheet ABCDCatering.xls")
        sys.exit(1)
    
    ws = wb.Sheets('Report')
    xldata = ws.UsedRange.Value
    newdata = []
    
    for row in xldata:
        if len(row) == 13 and row[-1] is not None:
            newdata.append(list(row))
            
    lasthdr = ""
    for i,field in enumerate(newdata[0]):
        
        if field is None:
            newdata[0][i] = lasthdr + " Name"
        else:
            lasthdr = newdata[0][i]
            
    rowcnt = len(newdata)
    colcnt = len(newdata[0])
    wsnew = wb.Sheets.Add()
    wsnew.Name = 'RAW_DATA'
    wsnew.Range(wsnew.Cells(1,1),wsnew.Cells(rowcnt,colcnt)).Value = newdata
    wsnew.Columns.AutoFit()

    src = "%s!R1C1:R%dC%d"%(wsnew.Name,rowcnt,colcnt)
    
    # What were the total sales in each of the last four quarters?
    addpivot(wb,src, title="Sales by Quarter", filters=(), columns=(),rows=("Fiscal Quarter",),sumvalue="Sum of Net Booking",sortfield=())

    # What are the sales for each food item in each quarter?
    addpivot(wb,src,title="Sales by Food Item",filters=(),columns=("Food Name",),rows=("Fiscal Quarter",),sumvalue="Sum of Net Booking",sortfield=())

    # Who were the top 10 customers for ABCD Catering in 2009?
    addpivot(wb,src,title="Top 10 Customers",filters=(),columns=(),rows=("Company Name",),sumvalue="Sum of Net Booking",sortfield=("Company Name",win32c.xlDescending))

    # Who was the highest producing sales rep for the year?
    addpivot(wb,src,title="Top Sales Reps",filters=(),columns=(),rows=("Sales Rep Name","Company Name"),sumvalue="Sum of Net Booking",sortfield=("Sales Rep Name",win32c.xlDescending))

    # What food item had the highest unit sales in Q4?
    ptname = addpivot(wb,src,title="Unit Sales by Food",filters=("Fiscal Quarter",),columns=(),rows=("Food Name",),sumvalue="Sum of Quantity",sortfield=("Food Name",win32c.xlDescending))
    wb.Sheets("Unit Sales by Food").PivotTables(ptname).PivotFields("Fiscal Quarter").CurrentPage = "2009-Q4"

    if int(float(excel.Version)) >= 12:
        wb.SaveAs('newABCDCatering.xlsx',win32c.xlOpenXMLWorkbook)
    else:
        wb.SaveAs('newABCDCatering.xls')
    excel.Application.Quit()

if __name__ == "__main__":
    runexcel()


http://pythonexcels.com/extending-pivot-table-data/
http://pythonexcels.com/a-user-friendly-experience/



more using win32 please check [this](http://timgolden.me.uk/python/win32_how_do_i.html) link